In [1]:
import os

import bk.load
import bk.compute
import bk.plot
import bk.signal

import numpy as np
import pandas as pd

import scipy.signal
import scipy.stats
import scipy.io

import matplotlib.pyplot as plt
import matplotlib as mpl
import neuroseries as nts


import itertools
from tqdm import tqdm



In [2]:
def training_intervals():
    run_ = scipy.io.loadmat('runintervals.mat')['runintervals']
    runintervals = nts.IntervalSet(run_[:,0],run_[:,1],time_units='s')
    return runintervals

In [136]:
def mean_firing(neurons,intervals):
    binSize = 0.25
    t,fr = bk.compute.binSpikes(neurons,binSize)
    t = nts.Ts(t,time_units='s')
    fr = np.mean(fr[:,intervals.in_interval(t)>= 0]/binSize,0)
    t = t[intervals.in_interval(t)>=0]
    fr = nts.Tsd(t.index.values, d = fr)
    fr = bk.compute.nts_smooth(fr,1000,10000)
    return fr

In [137]:
bk.load.current_session()
neurons,metadata = bk.load.loadSpikeData(bk.load.path)
states = bk.load.states()
pos = bk.load.pos()
train = training_intervals()
pos = pos.restrict(train)
speed = bk.compute.old_speed(pos,1000)

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


In [138]:
%matplotlib qt
stru = ['Hpc','BLA']
types = ['Pyr','Int']

for s,t in itertools.product(stru,types):
    fr = mean_firing(neurons[(metadata.Region == s) & (metadata.Type == t)],train)
    plt.scatter(speed.realign(fr).values,fr.values,1)
    print(scipy.stats.linregress(fr,speed.realign(fr)))
# plt.legend(list(itertools.product(stru,types)))

# plt.subplot(2,1,2,sharex = sub)
# plt.plot(speed)

LinregressResult(slope=5.219820921800578, intercept=0.6089947800943141, rvalue=0.27088253522539074, pvalue=3.448800487213615e-248, stderr=0.15218614262135033)
LinregressResult(slope=0.27576530741510263, intercept=2.3013015936075947, rvalue=0.17209519424643574, pvalue=3.926540396109536e-99, stderr=0.012950650559154333)
LinregressResult(slope=-6.873217754870727, intercept=7.24936966146224, rvalue=-0.1519936260069324, pvalue=1.7089639195219226e-77, stderr=0.36669786256299663)
LinregressResult(slope=-1.0484954488719924, intercept=10.188188400014287, rvalue=-0.34408257811580123, pvalue=0.0, stderr=0.023474182670647506)


In [139]:
fr = mean_firing(neurons[(metadata.Region == 'Hpc') & (metadata.Type == 'Pyr')],train)
plt.scatter(speed.realign(fr).values,fr.values,2)
a = (scipy.stats.linregress(speed.realign(fr),fr))
x = np.linspace(0,20,2500)
y = a.intercept + x*a.slope
plt.plot(x,y,'red')
plt.ylabel('Taux de décharge')
plt.xlabel('Vitesse')

Text(0.5, 0, 'Vitesse')

In [94]:
plt.plot(speed)

In [83]:
plt.plot(speed.realign(fr))
plt.plot(speed)

In [78]:
0.342077765703416**2

0.11701719778864118

Int64Index([     500000,     1500000,     2500000,     3500000,     4500000,
                5500000,     6500000,     7500000,     8500000,     9500000,
            ...
            22669500000, 22670500000, 22671500000, 22672500000, 22673500000,
            22674500000, 22675500000, 22676500000, 22677500000, 22678500000],
           dtype='int64', name='Time (us)', length=3713)

In [52]:
type(mean_firing)

numpy.ndarray

In [128]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [131]:
bk.plot.intervals(states['Rem'])
plt.savefig('file.svg')

In [43]:
mean_firing.as_units('s')


AttributeError: 'numpy.ndarray' object has no attribute 'as_units'

In [34]:
(metadata.Type == t) * metadata.Region == 'Hpc'

0       True
1       True
2       True
3       True
4       True
       ...  
247    False
248    False
249    False
250    False
251    False
Length: 252, dtype: bool